In [63]:
import numpy as np
from sklearn.datasets import fetch_mldata
import linalg
from importlib import reload
import _pickle as cPickle
import scipy.linalg
from sklearn import metrics

reload(linalg)

<module 'linalg' from '/data/vaishaal/pictureweb/pywren_kernels/linalg.py'>

In [2]:
def unpickle(infile):
    fo = open(infile, 'rb')
    outdict = cPickle.load(fo)
    fo.close()
    return outdict

def load_cifar(path):
    train_batches = []
    train_labels = []
    for i in range(1,6):
        cifar_out = unpickle("{0}/data_batch_{1}".format(path, i))
        train_batches.append(cifar_out["data"])
        train_labels.extend(cifar_out["labels"])

    # Stupid bull shit to get pixels in correct order
    X_train= np.vstack(tuple(train_batches))
    y_train = np.array(train_labels)
    cifar_out = unpickle("../cifar/test_batch")
    X_test = cifar_out["data"]
    y_test = cifar_out["labels"]
    return (X_train, np.array(y_train)), (X_test, np.array(y_test))


In [5]:
mnist = fetch_mldata('MNIST original', data_home="/data/vaishaal/mnist")
X_train_mnist = mnist.data[:60000, :]/255.0
y_train_mnist = mnist.target[:60000]
X_test_mnist = mnist.data[60000:, :]/255.0
y_test_mnist = mnist.target[60000:]
y_train_mnist_one_hot = np.eye(10)[y_train_mnist.astype('int')]
%time K_train_mnist = linalg.computeRBFGramMatrix(X_train_mnist, X_train_mnist, gamma=1e-3)
%time K_test_mnist = linalg.computeRBFGramMatrix(X_test_mnist, X_train_mnist, gamma=1e-3)

CPU times: user 5min 3s, sys: 2min 7s, total: 7min 10s
Wall time: 1min 58s
CPU times: user 1min 36s, sys: 26.7 s, total: 2min 3s
Wall time: 17 s


In [21]:
cifar = np.load("/data/vaishaal/cifar.npz")
X_train_cifar = cifar["X_train"]/255.0
X_test_cifar = cifar["X_test"]/255.0
y_train_cifar = cifar["y_train"]
y_test_cifar = cifar["y_test"]
y_train_cifar_one_hot =  np.eye(10)[y_train_cifar.astype('int')] 
%time K_train_cifar = linalg.computeRBFGramMatrix(X_train_cifar, X_train_cifar, gamma=1e-2)
%time K_test_cifar = linalg.computeRBFGramMatrix(X_test_cifar, X_train_cifar, gamma=1e-2)


CPU times: user 9min 23s, sys: 3min 12s, total: 12min 36s
Wall time: 1min 29s
CPU times: user 3min 31s, sys: 1min 10s, total: 4min 42s
Wall time: 17.7 s


In [7]:
def direct_solve(K, y, lambdav):
   diag = np.diag_indices(K.shape[0])
   K[diag] += lambdav
   model = scipy.linalg.solve(K, y)
   K[diag] -= lambdav
   return model

def evaluate(K, model, y):
    y_hat= np.argmax(K.dot(model), axis=1)
    return metrics.accuracy_score(y, y_hat)


    

In [ ]:
%time model_direct_mnist = direct_solve(K_train_mnist, y_train_mnist_one_hot, 1e-8)

In [13]:
%time train_acc_direct_mnist = evaluate(K_train_mnist, model_direct_mnist, y_train_mnist)
%time test_acc_direct_mnist = evaluate(K_test_mnist, model_direct_mnist, y_test_mnist)

CPU times: user 1min, sys: 284 ms, total: 1min 1s
Wall time: 938 ms
CPU times: user 9.06 s, sys: 32 ms, total: 9.1 s
Wall time: 126 ms


In [14]:
print(test_acc_direct_mnist)

0.9831


In [ ]:
%time model_direct_cifar = direct_solve(K_train_cifar, y_train_cifar_one_hot, 1e-8)

In [67]:
%time train_acc_direct_cifar = evaluate(K_train_cifar, model_direct_cifar, y_train_cifar)
%time test_acc_direct_cifar = evaluate(K_test_cifar, model_direct_cifar, y_test_cifar)

CPU times: user 27.8 s, sys: 308 ms, total: 28.1 s
Wall time: 404 ms
CPU times: user 3.87 s, sys: 68 ms, total: 3.94 s
Wall time: 54.8 ms


In [69]:
print(test_acc_direct_cifar)

0.5665


In [ ]:
prc_mnist = linalg.make_smart_precondition(X_train_mnist, 1e-4, 1e-3, 8192)

In [94]:
%time prc_cifar = linalg.make_smart_precondition(X_train_cifar, 1e-4, 1e-2, 4096)

Computing Nystroem svd
Computing Nystroem features
Computing ZTZ
Computing Cholesky
(4096, 50000)
CPU times: user 24min 18s, sys: 2min 15s, total: 26min 34s
Wall time: 41.8 s


In [97]:
%time model_pcg_cifar = linalg.pcg(K_train_cifar, y_train_cifar_one_hot, max_iter=100, tol=1e-5, lambdav=1e-4)

0.000995371595108
Iter 0, NORM IS 281.0561590389708
0.00280952693863
Iter 1, NORM IS 458.94424287103965
0.00135536973015
Iter 2, NORM IS 643.4149922932044
0.00109003593424
Iter 3, NORM IS 762.0641410524732
0.00176839425423
Iter 4, NORM IS 919.2370460701544
0.000561681126682
Iter 5, NORM IS 844.5864711997222
0.00243096097127
Iter 6, NORM IS 893.8021504631239
0.000579240972722
Iter 7, NORM IS 853.1746244101008
0.00183048106724
Iter 8, NORM IS 771.072171135036
0.00099669672609
Iter 9, NORM IS 847.0881254425855
0.0008978641174
Iter 10, NORM IS 896.7963358117717
0.00154595825284
Iter 11, NORM IS 746.4168985187442
0.00189214294257
Iter 12, NORM IS 1123.0637079420392
0.000742916190447
Iter 13, NORM IS 1001.3000069246347
0.000853577643313
Iter 14, NORM IS 803.4151298761021
0.00271218078965
Iter 15, NORM IS 1306.2262822106743
0.000647495333109
Iter 16, NORM IS 1159.4100003184444
0.000612655099417
Iter 17, NORM IS 902.119136566129
0.00209164170873
Iter 18, NORM IS 1302.2767590647886
0.0007214611

KeyboardInterrupt: 

In [96]:
%time train_acc_pcg_cifar = evaluate(K_train_cifar, model_pcg_cifar, y_train_cifar)
%time test_acc_pcg_cifar = evaluate(K_test_cifar, model_pcg_cifar, y_test_cifar)
print(test_acc_pcg_cifar)

CPU times: user 27.3 s, sys: 340 ms, total: 27.6 s
Wall time: 406 ms
CPU times: user 4.55 s, sys: 440 ms, total: 4.99 s
Wall time: 74.5 ms
0.5661


In [39]:
model_pcg_mnist = linalg.pcg(K_train_mnist, y_train_mnist_one_hot, prc=prc_mnist, lambdav=1e-4)

0.0855163514671
Iter 0, NORM IS 244.20192899800213
0.0486306175857
Iter 1, NORM IS 213.21553555310464
0.0453362813533
Iter 2, NORM IS 184.57846177917
0.0380738702256
Iter 3, NORM IS 161.2357135596191
0.0314042059285
Iter 4, NORM IS 139.4982115754883
0.0355553084862
Iter 5, NORM IS 117.92797345225868
0.0384316327012
Iter 6, NORM IS 100.66339097106906
0.0333221046267
Iter 7, NORM IS 84.96019598941743
0.037548573713
Iter 8, NORM IS 71.31981412668209
0.0405266354378
Iter 9, NORM IS 59.13917291111017
0.0400532896607
Iter 10, NORM IS 49.080721905707904
0.0348853177677
Iter 11, NORM IS 41.6437744114345
0.03206649756
Iter 12, NORM IS 34.783567043373495
0.0356270085015
Iter 13, NORM IS 28.623954872205662
0.038632907667
Iter 14, NORM IS 23.260760932890726
0.0401483240024
Iter 15, NORM IS 18.90420870436777
0.038482213799
Iter 16, NORM IS 15.393016272043223
0.0361625470453
Iter 17, NORM IS 12.513839829481004
0.0346380342709
Iter 18, NORM IS 10.196739944377219
0.035337690254
Iter 19, NORM IS 8.1967

In [40]:
%time train_acc_pcg_mnist = evaluate(K_train_mnist, model_pcg_mnist, y_train_mnist)
%time test_acc_pcg_mnist = evaluate(K_test_mnist, model_pcg_mnist, y_test_mnist)

CPU times: user 51.3 s, sys: 388 ms, total: 51.7 s
Wall time: 848 ms
CPU times: user 9.11 s, sys: 504 ms, total: 9.62 s
Wall time: 140 ms


In [42]:
print(train_acc_pcg_mnist)

0.999966666667


In [58]:
model_pcg_cifar = linalg.pcg(K_train_cifar, y_train_cifar_one_hot, prc=prc_cifar, lambdav=1e-4)

0.00017409246026
Iter 0, NORM IS 284.08201967338914
0.000122872555648
Iter 1, NORM IS 215.79621321939052
0.000136680688995
Iter 2, NORM IS 216.26059479848493
0.000138040305048
Iter 3, NORM IS 186.79883062545656
0.000142030801601
Iter 4, NORM IS 186.22054881794165
0.000143432622713
Iter 5, NORM IS 175.95061061362492
0.000146436768411
Iter 6, NORM IS 172.30354103367833
0.000148560556209
Iter 7, NORM IS 174.12368599463989
0.000148306946005
Iter 8, NORM IS 169.57799692608785
0.000150718120911
Iter 9, NORM IS 171.46451416047557
0.000152245083861
Iter 10, NORM IS 169.0583766719635
0.000153901532131
Iter 11, NORM IS 168.9650226926668
0.000155426206766
Iter 12, NORM IS 169.0850222117806
0.00015921972123
Iter 13, NORM IS 168.50749860058275
0.000164970605135
Iter 14, NORM IS 169.1590278858821
0.00016941356606
Iter 15, NORM IS 167.75970462328814
0.000172534256313
Iter 16, NORM IS 169.54466860261238
0.000176028298486
Iter 17, NORM IS 167.70860944293887
0.0001804574137
Iter 18, NORM IS 168.37625169

In [59]:
%time train_acc_pcg_cifar = evaluate(K_train_cifar, model_pcg_cifar, y_train_cifar)
%time test_acc_pcg_cifar = evaluate(K_test_cifar, model_pcg_cifar, y_test_cifar)

CPU times: user 28.6 s, sys: 368 ms, total: 28.9 s
Wall time: 423 ms
CPU times: user 5.1 s, sys: 408 ms, total: 5.51 s
Wall time: 77.4 ms


In [60]:
print(test_acc_pcg_cifar)

0.5303
